# Practical part for the PDC project

In [ ]:
import numpy as np
import scipy as sp

## Functions handling bit vectors

In [ ]:
# Performs a conversion from ASCII string to binary symbols
def ascii_str_to_binary(ascii_str):
    return str(''.join(format(ord(i), '08b')[1:] for i in ascii_str))

# Performs the inverse conversion, from a binary stream to ASCII characters
def binary_to_ascii_str(binstr):
    byte_binstr = "".join([f"0{byte}" for byte in split_bit_str(binstr, 7)])
    binary_int = int(byte_binstr, 2)
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    return binary_array.decode()

# Splits a bit string composed of 7-bit codewords into chunks of size 'chunk_size'.
def split_bit_str(bit_str, chunk_size):
    return [bit_str[i:i+chunk_size] for i in range(0, len(bit_str), chunk_size)]

# Gets the whole alphabet from a codeword length
def get_alphabet_from_codeword_length(length):
    return [f'{i:0{length}b}' for i in range(0, 2**length)]

## Decoding criterion functions

In [ ]:
# Computes the distance between a symbol and every point of a given constellation and returns the index of the nearest point in the constellation
def minimum_distance_criterion(constellation, symbol):
    return np.argmin([((symbol.real - point.real)**2 + (symbol.imag - point.imag)**2) for point in constellation])

## Debugging functions to check the encoding/decoding states

In [ ]:
# Gets every possible codeword lengths for which the length divides the total length of the original bitstring
def compute_codeword_lengths(bit_str):
    n = len(bit_str)
    return [i for i in range(1, n+1) if n % i == 0]

# Gets all the positions where bit_str1 is different from bit_str2 (bit_str1 and bit_str2 MUST HAVE THE SAME SIZE !)
def get_diff_positions(bit_str1, bit_str2):
    return [i for i in range(0, len(bit_str1)) if bit_str1[i] != bit_str2[i]]

## Constellation-related functions

### m-PSK

In [ ]:
# Performs the m-PSK encoding
def m_psk_encoder(energy_per_symbol, codeword, alphabet, constellation = [], d=0):
    k = alphabet.index(codeword)
    m = len(alphabet)
    return np.sqrt(energy_per_symbol) * np.exp(2j * np.pi * (k/m))

### m-QAM

In [ ]:
# Computes the PAM constellation for a given number of points and a distance for each of these points.
def pam(n, d):
    right_side = np.array([d*i + (d/2) for i in range(0, int(n/2))])
    left_side = -right_side
    return np.append(np.sort(left_side), right_side)

# Computes the whole m-QAM constellation, to be used to map each codeword to a point generated here
# WARNING: m MUST BE A POWER OF 2 HERE !!!
def m_qam(m, d, max_energy = (3/2)):
    if d > max_energy:
        raise OverflowError("ERROR: the distance you have set is too big for the project.")

    axis_len = int(np.sqrt(m))
    if axis_len % 2 != 0:
        axis_len = axis_len + 1

    diff_with_nearest_square = m - axis_len**2

    # if the difference is positive => we need to add some more points
    if diff_with_nearest_square > 0:
        real_axis_pam = pam(diff_with_nearest_square, d)
    else:
        real_axis_pam = pam(axis_len, d)
    if real_axis_pam.max() > max_energy:
        raise OverflowError("ERROR: You have some points that are outside the constrained square. Try to lower the distance or reduce the size of the constellation.")
    
    imaginary_axis_pam = pam(axis_len, d)[::-1]
    points_pairs = np.array([[complex(re, im) for re in real_axis_pam] for im in imaginary_axis_pam])

    # if the difference is negative => we need to remove some points from the constellation
    if diff_with_nearest_square < 0:
        nb_elts_to_delete_per_half_row = np.sqrt(-diff_with_nearest_square)/2
        nb_rows_to_modify = -diff_with_nearest_square / (2*nb_elts_to_delete_per_half_row)

        # First we set them to 0
        for j in range(0, int(nb_rows_to_modify/2)):
            for i in range(0, int(nb_elts_to_delete_per_half_row)):
                points_pairs[j, i] = 0
                points_pairs[j, len(points_pairs)-1-i] = 0
                points_pairs[len(points_pairs)-1-j, i] = 0
                points_pairs[len(points_pairs)-1-j, len(points_pairs)-1-i] = 0

    # We flatten the array to make it easier to map the codewords
    points_pairs = points_pairs.flatten()

    # Then, we remove all the points set to 0 (impossible to have if the dimension is a full square, so no worries)
    points_pairs = np.delete(points_pairs, np.argwhere(points_pairs == 0.0+0.0j))

    return points_pairs


# Performs the QAM encoding (in a linear fashion for the moment => might be re-worked)
def m_qam_encoder(codeword, constellation, alphabet, energy_per_symbol=0):
    try:
        k = alphabet.index(codeword)
        m = len(alphabet)
        return constellation[k]
    except OverflowError as ovferr:
        raise

## Encoder part

In [ ]:
# Encodes a string into a sequence of complex numbers & adds dummy points in front of it, given an encoder, a constellation, an alphabet and a number of dummy points
def encode_string(raw_str, n_dummy_symbols, encoder, dummy_symbol, constellation = [], alphabet = []):
    if n_dummy_symbols >= 100:
        raise OverflowError("ERROR: The batch of dummy samples cannot be equal or exceed 100 symbols.") 
    bit_str = ascii_str_to_binary(raw_str)
    splitted_bit_str = split_bit_str(bit_str, int(np.log2(len(alphabet))))
    splitted_bit_str_size = len(splitted_bit_str)
    if splitted_bit_str_size > 100:
        raise OverflowError("ERROR: The string without the dummy symbols cannot exceed 100 symbols.") 

    if splitted_bit_str_size + n_dummy_symbols > 100:
        raise OverflowError(f"ERROR: the number of dummy samples is too high, the max value you can set here is {100 - splitted_bit_str_size}.")
            
    theta_estimator_batch = np.full((n_dummy_symbols, 1), dummy_symbol)

    return np.append(theta_estimator_batch, np.array([encoder(codeword=codeword, constellation=constellation, alphabet=alphabet) for codeword in splitted_bit_str]))

## Channel part (given)

In [ ]:
def channel(sent_signal):
    s = np.mean(sent_signal**2)
    if s <= 1:
        s = 1
    noise_power = (10**(-2.65))*s
    shift = np.exp(-2j*np.pi*np.random.rand())
    # print(f"Applied shift: {np.angle(shift)}")
    sent_signal = sent_signal*shift
    noise_std = np.sqrt(noise_power/2)
    rcv_signal = sent_signal + noise_std*np.random.randn(len(sent_signal)) + 1j*noise_std*np.random.randn(len(sent_signal))
    return rcv_signal

## Decoder part

In [ ]:
# Decodes a bitstring outputted by the channel, given a constellation, an alphabet, a dummy symbol and the number of sent dummy symbols.

def decode_str(noisy_str, dummy_symbol, n_dummy_symbols, alphabet, constellation):
    # Isolate the dummy symbols to estimate the phase shift
    dummy_symbols = noisy_str[0:n_dummy_symbols]
    
    # Estimate the phase shift (np.angle gives a value between (-pi, pi])
    phase = (np.angle(np.sum(dummy_symbols)) + np.pi) - (np.angle(dummy_symbol) + np.pi) 

    # print(f"Estimated phase: {phase}, phase with normalization: {phase % 2*np.pi}")
    # Apply the inverse phase shift to every codeword
    dephased_symbols = np.exp(-1j * phase) * noisy_str[n_dummy_symbols:]

    decoded_codewords = [alphabet[minimum_distance_criterion(constellation, symbol)] for symbol in dephased_symbols]
    resulting_bit_str = "".join(decoded_codewords)
    return resulting_bit_str

## Putting everything together

In [ ]:
# 78-chars ASCII string -> TO BE CHANGED FOR EACH 
str_to_process = "hellohowareyoutodayimtryingtodecodethestringliketheonegivenattheexambutitshard"

max_energy = 3/2
encoder = m_qam_encoder
dummy_symbol = max_energy + 1j * max_energy
original_bitstr = ascii_str_to_binary(str_to_process)

# TODO: VARY BOTH DISTANCE_RANGE & N_DUMMY_SYMBOLS TO FIND BEST PARAMETERS (i.e., those for which the decoding is perfect)
distance_range = np.arange(0.15, max_energy, 0.01) # (start_value, max_value, step_increment_value)
n_dummy_symbols = 9

best_distance_values_per_qam_configuration = dict()

print(f"Performing an encoding/decoding on the string {str_to_process}:")
for codeword_size in compute_codeword_lengths(ascii_str_to_binary(str_to_process)):
    # We limit the size of codewords to avoid unrealistic constellations given our max-energy constraint
    if 2 <= codeword_size and codeword_size <= 10:
        ideal_distances = []
        for d in distance_range:
            try:
                constellation = m_qam(2**codeword_size, d)
                alphabet = get_alphabet_from_codeword_length(codeword_size)

                # We encode the string
                encoded_str = encode_string(str_to_process, constellation=constellation, alphabet=alphabet, n_dummy_symbols=n_dummy_symbols, dummy_symbol=dummy_symbol, encoder=encoder)

                # We pass it to the channel
                noisy_result = channel(encoded_str)

                # We decode it finally
                decoded_bitstr = decode_str(noisy_result, dummy_symbol=dummy_symbol, n_dummy_symbols=n_dummy_symbols, alphabet=alphabet, constellation=constellation)
                diff_positions = get_diff_positions(original_bitstr, decoded_bitstr)

                if len(diff_positions) == 0:
                    ideal_distances.append(d)      
            except OverflowError as ovferr:
                break # Go to another QAM configuration when we reach/go past the max-energy limit
            except UnicodeDecodeError as unicodeerr:
                continue

        best_distance_values_per_qam_configuration[f"{2**codeword_size}-QAM"] = ideal_distances

# For each valid QAM configuration : print the best distance values
for key, value in best_distance_values_per_qam_configuration.items():
    print(key)
    print(f" -> {value}")